<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Further_Enhancements_and_Techniques_for_Expert_Level_Projects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner --upgrade

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import VGG16
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

# Load the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images to values between 0 and 1
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Reshape data to fit model input shape (28, 28, 1)
x_train_cnn = x_train.reshape((60000, 28, 28, 1))
x_test_cnn = x_test.reshape((10000, 28, 28, 1))

# Create a validation set from the training data
x_train_cnn, x_val_cnn = x_train_cnn[:-12000], x_train_cnn[-12000:]
y_train, y_val = y_train[:-12000], y_train[-12000:]

# Create a sequential model
model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model_cnn.add(layers.MaxPooling2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPooling2D((2, 2)))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))) # Regularization
model_cnn.add(layers.Dropout(0.5))  # Dropout regularization
model_cnn.add(layers.Dense(10, activation='softmax'))

# Compile the model
model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False
)
datagen.fit(x_train_cnn)

# Learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train the model with augmented data and learning rate scheduler
model_cnn.fit(datagen.flow(x_train_cnn, y_train, batch_size=32), epochs=5, validation_data=(x_val_cnn, y_val), callbacks=[lr_scheduler])

# Evaluate the model
test_loss_cnn, test_acc_cnn = model_cnn.evaluate(x_test_cnn, y_test)
print(f'Test accuracy: {test_acc_cnn}')

# Make predictions
predictions_cnn = model_cnn.predict(x_test_cnn)

# Plotting some test images with their predicted labels
for i in range(5):
    plt.imshow(x_test_cnn[i].reshape(28, 28), cmap='gray')
    plt.title(f'Predicted label: {np.argmax(predictions_cnn[i])}')
    plt.axis('off')
    plt.show()

# Resize MNIST images to 224x224 to match VGG16 input shape
def resize_images(images):
    resized_images = np.zeros((images.shape[0], 224, 224, 3))
    for i in range(images.shape[0]):
        img = array_to_img(images[i].reshape(28, 28, 1))
        img = img.resize((224, 224))
        img = img.convert("RGB") # Convert grayscale to RGB
        resized_images[i] = img_to_array(img)
    return resized_images / 255.0  # Normalize to [0, 1]

x_train_vgg = resize_images(x_train_cnn)
x_test_vgg = resize_images(x_test_cnn)
x_val_vgg = resize_images(x_val_cnn)

# Transfer Learning with VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top of the base model
model_transfer = models.Sequential()
model_transfer.add(base_model)
model_transfer.add(layers.Flatten())
model_transfer.add(layers.Dense(256, activation='relu'))
model_transfer.add(layers.Dense(10, activation='softmax'))

# Compile and train the new model
model_transfer.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_transfer.fit(x_train_vgg, y_train, epochs=5, validation_data=(x_val_vgg, y_val))

# Hyperparameter Tuning with Keras Tuner
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(hp.Int('conv_units', min_value=32, max_value=128, step=32), (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('dense_units', min_value=64, max_value=256, step=64), activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5)

tuner.search(x_train_cnn, y_train, epochs=5, validation_data=(x_val_cnn, y_val))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras_tuner.tuners import RandomSearch

# Load the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images to values between 0 and 1
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Reshape data to fit model input shape (28, 28, 1)
x_train_cnn = x_train.reshape((60000, 28, 28, 1))
x_test_cnn = x_test.reshape((10000, 28, 28, 1))

# Create a validation set from the training data
x_train_cnn, x_val_cnn = x_train_cnn[:-12000], x_train_cnn[-12000:]
y_train, y_val = y_train[:-12000], y_train[-12000:]

# Create a sequential model
model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model_cnn.add(layers.MaxPooling2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPooling2D((2, 2)))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))) # Regularization
model_cnn.add(layers.Dropout(0.5))  # Dropout regularization
model_cnn.add(layers.Dense(10, activation='softmax'))

# Compile the model
model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False
)
datagen.fit(x_train_cnn)

# Learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train the model with augmented data and learning rate scheduler
model_cnn.fit(datagen.flow(x_train_cnn, y_train, batch_size=32), epochs=5, validation_data=(x_val_cnn, y_val), callbacks=[lr_scheduler])

# Evaluate the model
test_loss_cnn, test_acc_cnn = model_cnn.evaluate(x_test_cnn, y_test)
print(f'Test accuracy: {test_acc_cnn}')

# Make predictions
predictions_cnn = model_cnn.predict(x_test_cnn)

# Plotting some test images with their predicted labels
for i in range(5):
    plt.imshow(x_test_cnn[i].reshape(28, 28), cmap='gray')
    plt.title(f'Predicted label: {np.argmax(predictions_cnn[i])}')
    plt.axis('off')
    plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images to values between 0 and 1
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Reshape data to fit model input shape (28, 28, 1)
x_train_cnn = x_train.reshape((60000, 28, 28, 1))
x_test_cnn = x_test.reshape((10000, 28, 28, 1))

# Create an instance of ImageDataGenerator with augmentation options
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False
)
datagen.fit(x_train_cnn)

# Create a validation set from the training data
x_train_cnn, x_val_cnn = x_train_cnn[:-12000], x_train_cnn[-12000:]
y_train, y_val = y_train[:-12000], y_train[-12000:]

# Load a pre-trained VGG16 model without the top layer (the classification part)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top of the base model
model_transfer = models.Sequential()
model_transfer.add(layers.UpSampling2D(size=(8, 8), input_shape=(28, 28, 1)))  # Resizing input to match VGG16
model_transfer.add(layers.Conv2D(3, (3, 3), padding='same'))  # Matching channel number to 3
model_transfer.add(base_model)
model_transfer.add(layers.Flatten())
model_transfer.add(layers.Dense(256, activation='relu'))
model_transfer.add(layers.Dense(10, activation='softmax'))

# Compile and train the new model
model_transfer.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_transfer.fit(x_train_cnn, y_train, epochs=5, validation_data=(x_val_cnn, y_val))

# Evaluate the model
test_loss_transfer, test_acc_transfer = model_transfer.evaluate(x_test_cnn, y_test)
print(f'Test accuracy: {test_acc_transfer}')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from keras_tuner import RandomSearch

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images to values between 0 and 1
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Reshape data to fit model input shape (28, 28, 1)
x_train_cnn = x_train.reshape((60000, 28, 28, 1))
x_test_cnn = x_test.reshape((10000, 28, 28, 1))

# Define the model building function
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(hp.Int('conv_units', min_value=32, max_value=128, step=32), (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('dense_units', min_value=64, max_value=256, step=64), activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize the tuner
tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5)

# Perform the search
tuner.search(x_train_cnn, y_train, epochs=5, validation_split=0.2)

# Print the best hyperparameters found
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f'Best hyperparameters: Conv units: {best_hps.get("conv_units")}, Dense units: {best_hps.get("dense_units")}')